# 정령 시연 시나리오 (던전)  
## 1) 1층 던전 입장 (게임 안내, 전투전 정비)  
- "던전을 돌면 뭐가 좋다 했지?"
- "스킬 사용법이 뭐야?"
- "내 아이템중에 가장 쎈 무기가 뭐야?"

In [1]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_dungeon_agent import graph_builder as dungeon_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message
from core.game_dto.z_muck_factory import MockFactory
from agents.fairy.fairy_state import DungeonPlayerState

player = DungeonPlayerState(
    playerId=1,
    heroineId=1,
    currRoomId=1,
    difficulty=0,
    hp = 250,
    moveSpeed = 1,
    attackSpeed = 1.0,        
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)
memory = MemorySaver()
graph = dungeon_graph_builder.compile(memory)

config = {"configurable": {"thread_id": "user_1"}}
inputs = {
    "messages": [add_human_message(content="스킬 사용법이 뭐야?")],    
    "dungenon_player": player,
    "target_monster_ids":[0,1],
}
await graph.ainvoke(inputs, config=config)
result = await graph.ainvoke({
    "messages":[add_human_message("내 아이템중에 가장 쎈 무기가 뭐야?")]
}, config=config)
result["messages"]

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/pydantic/main.py:250: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


전체 의도:: intents=[<FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 4, 'neighbors': [2, 3, 0], 'monsters': [{'monster_id': 1, 'pos_x': 0.5399706363677979, 'pos_y': 0.2424842119216919}, {'monster_id': 1, 'pos_x': 0.8617395162582397, 'pos_y': 0.5734589099884033}], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 8, 'neighbors': [1, 4], 'monsters': [{'monster_id': 1, 'pos_x': 0.4251292943954468, 'pos_y': 0.17043542861938477}, {'monster_id': 1, 'pos_x': 0.17976415157318115, 'pos_y': 0.8153200149536133}], 'room_id': 2, 'room_type': 'monster', 'event_type': 0}, {'size': 6, 'neighbors': [1], 'monsters': [], 'room_id': 3, 'room_type': 'event', 'event_type': 1}, {'size': 12, 'neighbors': [2], 'monsters': [], 'room_id': 4, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [1], 'heroine_ids': [1]} balanced_map={'rooms': [{'size'

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:19.401086'}, response_metadata={}, id='a4c741e4-2bc1-4044-907a-2e592035d68a'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-10T11:15:21.052042', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='0a15ac6d-af4e-45f7-8e93-cb52bd0c41c5'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:21.052838'}, response_metadata={}, id='284f9b87-bd11-43a7-8869-4de703c7e2e3'),
 AIMessage(content='인벤토리 봤어! 레어 대검(공격력 25)이 제일 강력해. 장착해볼까?', additional_kwargs={'created_at': '2025-12-10T11:15:28.953126', 'intent_types': ['INTERACTION_HANDLER', 'USAGE_GUIDE']}, response_metadata={}, id='ebc94165-3c1b-41c2-9c03-82f22210440b')]

## 2)  1층 던전 첫 몬스터 조우 (몬스터 정보)
- "저기 있는 몬스터는 뭐야?"
- "내가 잡을 수 있지?"

In [2]:
result = await graph.ainvoke(
    {"messages": add_human_message(content="저기 있는 몬스터는 뭐야?")}, config=config
)

# result = await graph.ainvoke(
#     {
#         "messages": add_human_message(content="내가 잡을 수 있지?"),
#     },
#     config=config,
# )
result["messages"]

전체 의도:: intents=[<FairyDungeonIntentType.MONSTER_GUIDE: 'MONSTER_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 4, 'neighbors': [2, 3, 0], 'monsters': [{'monster_id': 1, 'pos_x': 0.5399706363677979, 'pos_y': 0.2424842119216919}, {'monster_id': 1, 'pos_x': 0.8617395162582397, 'pos_y': 0.5734589099884033}], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 8, 'neighbors': [1, 4], 'monsters': [{'monster_id': 1, 'pos_x': 0.4251292943954468, 'pos_y': 0.17043542861938477}, {'monster_id': 1, 'pos_x': 0.17976415157318115, 'pos_y': 0.8153200149536133}], 'room_id': 2, 'room_type': 'monster', 'event_type': 0}, {'size': 6, 'neighbors': [1], 'monsters': [], 'room_id': 3, 'room_type': 'event', 'event_type': 1}, {'size': 12, 'neighbors': [2], 'monsters': [], 'room_id': 4, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [1], 'heroine_ids': [1]} balanced_map={'rooms': [{'s

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:19.401086'}, response_metadata={}, id='a4c741e4-2bc1-4044-907a-2e592035d68a'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-10T11:15:21.052042', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='0a15ac6d-af4e-45f7-8e93-cb52bd0c41c5'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:21.052838'}, response_metadata={}, id='284f9b87-bd11-43a7-8869-4de703c7e2e3'),
 AIMessage(content='인벤토리 봤어! 레어 대검(공격력 25)이 제일 강력해. 장착해볼까?', additional_kwargs={'created_at': '2025-12-10T11:15:28.953126', 'intent_types': ['INTERACTION_HANDLER', 'USAGE_GUIDE']}, response_metadata={}, id='ebc94165-3c1b-41c2-9c03-82f22210440b'),
 HumanMessage(content='저기 있는 몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:28.961902'}, response_metadata={}, id='edd75297-72c7-4328-89d0-8a87c6dee6f0'),
 AIMessage(co

## 3) 1층 이벤트방 (이벤트 힌트 정보)
"이벤트 어떻게 깨?"

In [3]:
player = DungeonPlayerState(
    playerId=1,
    heroineId=1,
    currRoomId=3,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="이벤트 어떻게 깨?"),
        "dungenon_player": player,
    },

    config=config,
)
result["messages"]

# 전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
# expected_outcome='1번 선택은 기억의 고통을 정면으로 마주해 정신적 타격(instant_damage_medium)을 입지만, 트라우마를 전투 감각으로 전환해 스킬 데미지(skill_damage_increase_all)가 크게 상승한다.\n2번 선택은 죄책감을 회피해 즉각적인 피해는 피하지만, 마음이 무거운 족쇄가 되어 이동속도(move_speed_decrease_all)가 감소해 전투와 탐사에서 기동력이 떨어진다.\n3번 선택은 분노로 정신을 끊어내며 일시적 피해(instant_damage_low)를 받는 대신, 신경이 과도하게 흥분해 공격 속도(attack_speed_increase_all)가 빨라져 근접 교전에 유리해진다.\n4번 선택은 죄책감을 인정하고 상처를 들여다보며 내면이 단단해져 최대 체력(hp_increase_all)이 상승하지만, 기억의 잔향이 스킬 운용을 방해해 일정 시간 스킬 데미지 감소(curse_debuff)를 겪게 된다.') floor=1



전체 의도:: intents=[<FairyDungeonIntentType.EVENT_GUIDE: 'EVENT_GUIDE'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 4, 'neighbors': [2, 3, 0], 'monsters': [{'monster_id': 1, 'pos_x': 0.5399706363677979, 'pos_y': 0.2424842119216919}, {'monster_id': 1, 'pos_x': 0.8617395162582397, 'pos_y': 0.5734589099884033}], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 8, 'neighbors': [1, 4], 'monsters': [{'monster_id': 1, 'pos_x': 0.4251292943954468, 'pos_y': 0.17043542861938477}, {'monster_id': 1, 'pos_x': 0.17976415157318115, 'pos_y': 0.8153200149536133}], 'room_id': 2, 'room_type': 'monster', 'event_type': 0}, {'size': 6, 'neighbors': [1], 'monsters': [], 'room_id': 3, 'room_type': 'event', 'event_type': 1}, {'size': 12, 'neighbors': [2], 'monsters': [], 'room_id': 4, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [1], 'heroine_ids': [1]} balanced_map={'rooms': [{'size'

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:19.401086'}, response_metadata={}, id='a4c741e4-2bc1-4044-907a-2e592035d68a'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-10T11:15:21.052042', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='0a15ac6d-af4e-45f7-8e93-cb52bd0c41c5'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:21.052838'}, response_metadata={}, id='284f9b87-bd11-43a7-8869-4de703c7e2e3'),
 AIMessage(content='인벤토리 봤어! 레어 대검(공격력 25)이 제일 강력해. 장착해볼까?', additional_kwargs={'created_at': '2025-12-10T11:15:28.953126', 'intent_types': ['INTERACTION_HANDLER', 'USAGE_GUIDE']}, response_metadata={}, id='ebc94165-3c1b-41c2-9c03-82f22210440b'),
 HumanMessage(content='저기 있는 몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:28.961902'}, response_metadata={}, id='edd75297-72c7-4328-89d0-8a87c6dee6f0'),
 AIMessage(co

## 4) 1층 보스방 입장 전 (던전 정보)
"다음 방은 뭐가 나올까?"

In [4]:
player = DungeonPlayerState(
    playerId=1,
    heroineId=1,
    currRoomId=3,
    difficulty=0,
    hp=850,
    moveSpeed=1,
    attackSpeed=1.0,
    weapon=MockFactory.create_weapon(),
    inventory=[0, 21, 42],
)

result = await graph.ainvoke(
    {
        "messages": add_human_message(content="보스방으로 어떻게 가?"),
        "dungenon_player": player,
    },
    config=config,
)
result["dungenon_player"]
result["messages"]
# 전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>, <FairyDungeonIntentType.USAGE_GUIDE: 'USAGE_GUIDE'>]


전체 의도:: intents=[<FairyDungeonIntentType.DUNGEON_NAVIGATOR: 'DUNGEON_NAVIGATOR'>]
던전 로우 raw_map={'rooms': [{'size': 4, 'neighbors': [1], 'monsters': [], 'room_id': 0, 'room_type': 'empty', 'event_type': 0}, {'size': 4, 'neighbors': [2, 3, 0], 'monsters': [{'monster_id': 1, 'pos_x': 0.5399706363677979, 'pos_y': 0.2424842119216919}, {'monster_id': 1, 'pos_x': 0.8617395162582397, 'pos_y': 0.5734589099884033}], 'room_id': 1, 'room_type': 'monster', 'event_type': 0}, {'size': 8, 'neighbors': [1, 4], 'monsters': [{'monster_id': 1, 'pos_x': 0.4251292943954468, 'pos_y': 0.17043542861938477}, {'monster_id': 1, 'pos_x': 0.17976415157318115, 'pos_y': 0.8153200149536133}], 'room_id': 2, 'room_type': 'monster', 'event_type': 0}, {'size': 6, 'neighbors': [1], 'monsters': [], 'room_id': 3, 'room_type': 'event', 'event_type': 1}, {'size': 12, 'neighbors': [2], 'monsters': [], 'room_id': 4, 'room_type': 'boss', 'event_type': 0}], 'rewards': [], 'player_ids': [1], 'heroine_ids': [1]} balanced_map={'room

[HumanMessage(content='스킬 사용법이 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:19.401086'}, response_metadata={}, id='a4c741e4-2bc1-4044-907a-2e592035d68a'),
 AIMessage(content='스킬은 E키로 무기 스킬, R키로 직업 고유 스킬을 써! 재미있게 싸워봐~', additional_kwargs={'created_at': '2025-12-10T11:15:21.052042', 'intent_types': ['USAGE_GUIDE']}, response_metadata={}, id='0a15ac6d-af4e-45f7-8e93-cb52bd0c41c5'),
 HumanMessage(content='내 아이템중에 가장 쎈 무기가 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:21.052838'}, response_metadata={}, id='284f9b87-bd11-43a7-8869-4de703c7e2e3'),
 AIMessage(content='인벤토리 봤어! 레어 대검(공격력 25)이 제일 강력해. 장착해볼까?', additional_kwargs={'created_at': '2025-12-10T11:15:28.953126', 'intent_types': ['INTERACTION_HANDLER', 'USAGE_GUIDE']}, response_metadata={}, id='ebc94165-3c1b-41c2-9c03-82f22210440b'),
 HumanMessage(content='저기 있는 몬스터는 뭐야?', additional_kwargs={'created_at': '2025-12-10T11:15:28.961902'}, response_metadata={}, id='edd75297-72c7-4328-89d0-8a87c6dee6f0'),
 AIMessage(co

## 5) 3층 최종 보스 깨기전 
"기억의 조각을 얻으면 누구한테 찾아가면 돼?"

In [ ]:
result = await graph.ainvoke(
    { "messages": add_human_message(content="기억의 조각을 얻으면 누구한테 찾아가면 돼?"),
     "dungenon_player": player, },
    config=config
)
result["messages"]

# 정령 시연 시나리오 (길드)
## 1) 히로인과 대화시
"히로인의 호감도 얻는법 힌트좀 줘"

## 2) 해금 시나리오 가장 쉬운 단계 해제  후
"로코의 과거 이야기를 다시 듣고 싶어" 

In [ ]:
import os, sys
from pathlib import Path

src_path = Path(os.getcwd()).resolve().parents[1]  
sys.path.append(str(src_path))  
from agents.fairy.fairy_guild_agent import graph_builder as guild_graph_builder
from langgraph.checkpoint.memory import MemorySaver
from agents.fairy.util import add_human_message

memory = MemorySaver()
guild_graph = guild_graph_builder.compile(checkpointer=memory)
config = {
    "configurable": {"heroine_id": 3, 
                     "memory_progress": 90,
                    " affection":50, "sanity": 50, "thread_id": "100"}
}
result = await guild_graph.ainvoke(
    { "messages": add_human_message(content="히로인의 호감도 얻는 힌트좀 줘") },
    config=config
)
# result = await graph.ainvoke(
#     { "messages": add_human_message(content="로코의 과거 이야기를 다시 듣고 싶어")},
#     config=config
# )
result["messages"]